In [1]:
import requests
import pandas as pd
from datetime import *
url = "https://api3.binance.com/api/v3/"

In [5]:
%%time
start = datetime.strptime("20201231", "%Y%m%d")
start.strftime("%f")
data = list()
api = url + "klines"
while start < datetime.strptime("20220407", "%Y%m%d"):
    end = start + timedelta(days=10)
    start_time = int(datetime.timestamp(start) * 1000) 
    end_time = int(datetime.timestamp(end) * 1000)
    params = {"symbol":"AAVEUSDT", "limit":1000, "interval": "4h", "startTime": start_time, "endTime": end_time}
    res = requests.get(api, params)
    data.append(pd.DataFrame(res.json()))
    start = end

CPU times: user 331 ms, sys: 0 ns, total: 331 ms
Wall time: 18.5 s


In [6]:
eth_df = pd.concat(data)[[0, 4, 7, 10]].rename(columns={0:"TimeStamp", 4:"ClosePrice", 7:"TradeVolume", 10: "BuyVolume"})
eth_df["TimeStamp"] = pd.to_datetime(eth_df["TimeStamp"], unit="ms")
eth_df.set_index(["TimeStamp"], inplace=True)
for col in ["ClosePrice", "TradeVolume", "BuyVolume"]:
    eth_df[col] = eth_df[col].astype(float)
eth_df["Return"] = eth_df["ClosePrice"].shift(-1) / eth_df["ClosePrice"] - 1
eth_df["PriceTime"] = eth_df.index
eth_df

,ClosePrice,TradeVolume,BuyVolume,Return,PriceTime
TimeStamp,,,,,
2020-12-31 08:00:00,84.853,3.478623e+06,1.858921e+06,0.012810,2020-12-31 08:00:00
2020-12-31 12:00:00,85.940,5.154079e+06,2.915836e+06,0.005353,2020-12-31 12:00:00
2020-12-31 16:00:00,86.400,1.771539e+06,1.009712e+06,0.024074,2020-12-31 16:00:00
2020-12-31 20:00:00,88.480,3.034592e+06,1.710557e+06,-0.000294,2020-12-31 20:00:00
2021-01-01 00:00:00,88.454,3.870173e+06,1.966110e+06,-0.021491,2021-01-01 00:00:00
...,...,...,...,...,...
2022-04-09 12:00:00,177.400,3.925351e+06,2.090435e+06,0.029876,2022-04-09 12:00:00
2022-04-09 16:00:00,182.700,4.863981e+06,2.881364e+06,0.011494,2022-04-09 16:00:00
2022-04-09 20:00:00,184.800,2.162111e+06,1.194179e+06,-0.005952,2022-04-09 20:00:00


In [7]:
eth_df.to_csv("AAVE4h.csv", index_label="TimeStamp", sep="\t")

In [18]:
%%time
text = pd.read_csv("Data/TextData.csv", sep="\t", index_col="TimeStamp")[["ID", "Text", "AuthorID"]]
text.index = pd.to_datetime(text.index)

CPU times: user 2.88 s, sys: 126 ms, total: 3 s
Wall time: 3.49 s


In [19]:
df = pd.merge_asof(text, eth_df, left_index=True, right_index=True, direction="forward")

In [20]:
%%time
dataset = df.groupby(["PriceTime"]).mean()[["ClosePrice", "TradeVolume", "BuyVolume", "Return"]]
dataset["Text"] = df[["Text", "PriceTime"]].groupby(["PriceTime"]).agg(lambda x: " ".join(x))
# dataset = df[["ClosePrice", "TradeVolume", "BuyVolume", "Return", "Text", "PriceTime"]]
dataset.to_csv("Data/Dataset.csv", index_label="TimeStamp", sep="\t")

CPU times: user 2.6 s, sys: 92 ms, total: 2.69 s
Wall time: 3.79 s


In [21]:
df.iloc[1:20][["Text", "ClosePrice", "Return", "PriceTime"]]

,Text,ClosePrice,Return,PriceTime
TimeStamp,,,,
2021-01-01 00:02:00,$ETH Ranges for the next session Upper $746 07...,748.28,-0.005640,2021-01-01 01:00:00
2021-01-01 00:03:00,$ETH Ethereum today will give us an idea if it...,748.28,-0.005640,2021-01-01 01:00:00
2021-01-01 00:04:00,New GIVEAWAY Okcash $ok $BTC ETH 30 WINNERS 10...,748.28,-0.005640,2021-01-01 01:00:00
2021-01-01 00:04:00,New GIVEAWAY Okcash $ok $BTC ETH 30 WINNERS 10...,748.28,-0.005640,2021-01-01 01:00:00
2021-01-01 00:05:00,$ETH has out preformed $BTC by 53 in 2020 Bitc...,748.28,-0.005640,2021-01-01 01:00:00
2021-01-01 00:06:00,Wow Just wow Bitcoin Etheruem XRP BitcoinCash ...,748.28,-0.005640,2021-01-01 01:00:00
2021-01-01 00:12:00,Coin Mentions Spiked in reddit for $ETH 24HR P...,748.28,-0.005640,2021-01-01 01:00:00
2021-01-01 00:18:00,Prices update in $USD 1 hour $BTC 28859 5 $ 0...,748.28,-0.005640,2021-01-01 01:00:00
2021-01-01 00:19:00,1 ETH should be atleast $925 now $eth Ethereum...,748.28,-0.005640,2021-01-01 01:00:00
